In [ ]:
# Task 1: Establishing a connection

import mysql.connector as connector
from mysql.connector.pooling import MySQLConnectionPool
from mysql.connector import Error

# Database configurations
dbconfig = {"database" : "little_lemon_db", "user" : "root", "password" : ""}

try:
    pool = MySQLConnectionPool(pool_name = "pool_b",
    pool_size = 2,
    **dbconfig)
    print("The connection pool is created with a name: ", pool.pool_name)
    print("The pool size is:", pool.pool_size)
except Error as er:
    print("Error code:", er.errno)
    print("Error message:", er.msg)

In [ ]:
# Task 2: Add 2 connections and insert the following in the Bookings table:

# Connect the first guest.
guest_one = pool.get_connection()
cursor_one = guest_one.cursor()
booking_one = """INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(8,'Anees','Java','18:00:00',6);"""
cursor_one.execute(booking_one)
guest_one.commit()
print("""A new booking is added in the "Bookings" table.""")

# Connect the second guest .
guest_two = pool.get_connection()
cursor_two = guest_two.cursor()
booking_two = """INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(5, 'Bald','Vin','19:00:00',6);"""
cursor_two.execute(booking_two)
guest_two.commit()
print("""A new booking is added in the "Bookings" table.""")

# Adding a new connection to connect the third user.
import mysql.connector as connector
try:
    guest_three = pool.get_connection()
    print("The guest is connected")
except:
    print("Adding new connection in the pool.")
        
    # Create a connection
    connection = connector.connect(user= "root", password= "")
    # Add the connection into the pool
    pool.add_connection(cnx=connection)
    print("A new connection is added in the pool.\n")
        
    guest_three = pool.get_connection()
    print("'connection3' is added in the pool.")
    
# Connect the third guest 
guest_three = guest_three.cursor()
booking_three = """INSERT INTO Bookings 
(TableNo, GuestFirstName, GuestLastName, BookingSlot, EmployeeID)
VALUES
(12, 'Jay','Kon','19:30:00',6);"""
guest_three.execute(booking_three)
guest_three.commit()
print("""A new booking is added in the "Bookings" table.""")

# You can only return two connections back to the pool as the pool_size=2.
#  Close all  connections and use try-except to print the pool error if the pool is already full.

from mysql.connector import Error
for connection in [guest_one, guest_two, guest_three]:
    try:
        connection.close()
        print("Connection is returned to the pool")
    except Error as er:
        print("\nConnection can't be returned to the pool")
        print("Error message:", er.msg)

In [ ]:
# Close the open connections
guest_one.close()
guest_two.close()
guest_three.close()

In [ ]:
# Task 3: Create reports
# Return EmployeeID of Little Lemon Manager

ll_manager = """
SELECT EmployeeID AS ManagerID,
Name AS ManagerName
FROM Employees
WHERE Role = 'Manager';
"""
cursor.execute(ll_manager)
columns = cursor.column_names
print(columns)
results = cursor.fetchall()
for result in results:
    print(result)

In [ ]:
# Return the name and role of the employee who receives the highest salary

highest_salary = """
SELECT 
Name AS EmployeeName,
Role AS EmployeeRole
FROM Employees
ORDER BY Annual_Salary DESC LIMIT 1;
"""
cursor.execute(highest_salary)
columns = cursor.column_names
print(columns)
results = cursor.fetchall()
for result in results:
    print(result)

In [ ]:
# Return the number of guests booked between 18:00 and 20:00

six_and_eight = """
SELECT
COUNT(BookingID) AS BookingNumber
FROM Bookings
WHERE BookingSlot BETWEEN '18:00:00' AND '20:00:00';
"""
cursor.execute(six_and_eight)
columns = cursor.column_names
print(columns)
results = cursor.fetchall()
for result in results:
    print(result)

In [ ]:
# Return the full name and BookingID of all guests waiting to be seated with the receptionist in sorted with respect to their booking slot

waiting_guests = """
SELECT
Booking.BookingID AS ID,
CONCAT(Bookings.GuestFirstName, ' ', Bookings.GuestLastName) AS GuestName,
Role AS Employee
FROM Bookings 
LEFT JOIN 
Employees 
ON Employees.EmployeeID=Bookings.EmployeeID
WHERE Employees.Role = "Receptionist"
ORDER BY BookingSlot DESC;
"""

cursor.execute(waiting_guests)
columns = cursor.column_names
print(columns)
results = cursor.fetchall()
for result in results:
    print(result)

In [ ]:
# Task 4: Create a report:

connection = connector.connect(user = "root", password = "", database = "little_lemon_db")
cursor = connection.cursor()

sales_report = """ CREATE PROCEDURE BasicSalesReport()
BEGIN
SELECT SUM(BillAmount) AS TotalSales,
AVG(BillAmount) AS AverageSale,
MIN(BillAmount) AS Minimum_Bill_Paid,
MAX(BillAmount) AS Maximum_Bill_Paid
FROM Orders;
END"""

cursor.execute(sales_report)

cursor.callproc("BasicSalesReport")

results = next(cursor.stored_results())
results_two = results.fetchall()

for column_id in cursor.stored_results():
    cols = [ column[0] for column in column_id.description ]

print("Basic sales report:")
for result in results_two:
    print(cols[0],':',result[0])
    print(cols[1],':',result[1])
    print(cols[2],':',result[2])
    print(cols[3],':',result[3])

In [ ]:
# Task 5:  Display the next three upcoming bookings from the Bookings table on the kitchen screen to notify their chefs which orders are due next

connection_three = pool.get_connection()
buffered_cursor = connection.cursor(buffered=True)

next_three_bookings="""
SELECT 
Bookings.BookingSlot,
CONCAT(Bookings.GuestFirstName,' ',Bookings.GuestLastName) AS GuestName,
Employees.Name AS EmployeeName,
Employees.Role AS EmployeeRole
FROM Bookings 
INNER JOIN 
Employees ON Bookings.EmployeeID=Employees.EmployeeID
ORDER BY Bookings.BookingSlot ASC;
"""

buffered_cursor.execute(next_three_bookings)
results=buffered_cursor.fetchmany(size = 3)

for result in results:
    print("Booking Slot",result[0])
    print("Guest Name:",result[1])
    print("Assigned to:",result[2],"[{}]".format(result[3]))

connection_three.close()